In [23]:
# Importation des bibliothèques nécessaires
from rdflib import Graph

# Initialisation du graphe RDF
g = Graph()

# Chargement de l'ontologie
ontology_file = "dnd_ontology.rdf"
g.parse(ontology_file, format="xml")

# Fonction pour exécuter et afficher une requête SPARQL
def run_query(query):
    results = g.query(query)
    return results

def cleanIRI(iri):
    return f"\033[91m{iri.split('dnd#')[1]}\033[0m"

In [24]:
# 1. Récupérer tous les personnages et leur alignement
query1 = """
SELECT ?personnage ?alignement
WHERE {
  ?personnage a :Personnage .
  ?personnage :aPourAlignement ?alignement .
}
"""
results = run_query(query1)
for r in results:
  print(f"{cleanIRI(r.personnage)} a pour alignement {cleanIRI(r.alignement)}")

Aragorn a pour alignement LoyalBon
GrandMaitreArcanis a pour alignement VraiNeutre
LOmbreNoire a pour alignement ChaotiqueMauvais
RoiElron a pour alignement LoyalBon


In [25]:
# 2. Trouver les classes disponibles pour les personnages
query2 = """
SELECT DISTINCT ?classe
WHERE {
  ?type rdfs:subClassOf :Classe .
  ?classe a ?type .
}
"""
results = run_query(query2)
print("Classes disponibles :")
for r in results:
  print(f"- {cleanIRI(r.classe)}")

Classes disponibles :
- Assassin
- Barbare
- Guerrier
- Moine
- NonCombattant
- Barde
- Clerc
- Druide
- Magicien
- Occultiste
- Paladin
- Rodeur
- Sorcier


In [26]:
# 3. Lister les factions et toutes leurs informations
query3 = """
SELECT DISTINCT ?faction ?type ?alignement ?allies ?ennemis ?leader
WHERE {
  ?type rdfs:subClassOf :Faction .
  ?faction a ?type .
  ?faction :aPourAlignement ?alignement .
  OPTIONAL { ?faction a ?type . }
  OPTIONAL { ?faction :aPourAllies ?allies . }
  OPTIONAL { ?faction :aPourEnnemis ?ennemis . }
  OPTIONAL { ?faction :aPourLeader ?leader . }
  OPTIONAL { ?faction :aPourTerritoire ?territoire . }
}
"""
  
results = run_query(query3)
for r in results:
  print(f"{cleanIRI(r.faction)} est un.e {cleanIRI(r.type)}, a pour alignement {cleanIRI(r.alignement)}")
  if r.allies != None:
    print(f"- Alliés : {cleanIRI(r.allies)}")
  if r.ennemis != None:
    print(f"- Ennemis : {cleanIRI(r.ennemis)}")
  if r.leader != None:
    print(f"- Leader : {cleanIRI(r.leader)}")
    

LaGardeRoyale est un.e Armee, a pour alignement LoyalBon
- Alliés : LeConseilDesMages
- Ennemis : LesVoleursDeLOmbre
- Leader : RoiElron
LeConseilDesMages est un.e Guilde, a pour alignement VraiNeutre
- Alliés : LaGardeRoyale
- Leader : GrandMaitreArcanis
LesVoleursDeLOmbre est un.e OrganisationCriminelle, a pour alignement ChaotiqueMauvais
- Ennemis : LaGardeRoyale


In [27]:
# 4. Identifier les races et leur vitesse
query4 = """
SELECT ?race ?speed
WHERE {
  ?race rdfs:subClassOf :Race .
  ?race :aVitesseRace ?speed .
  
}
ORDER BY DESC(?speed)
"""
results = run_query(query4)
for r in results:
  print(f"{cleanIRI(r.race)} a pour vitesse {r.speed}")